# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [56]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nhl_correlation(): 
    # YOUR CODE HERE
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.rename(columns = {"Population (2016 est.)[8]":"Population"},inplace=True)
    #cities["NHL"] = cities["NHL"].str.replace("—", str(np.nan))
    cities["NHL"] = cities["NHL"].apply(lambda x: re.sub(r"\[.+\]", "", x))
    cities["NHL"] = cities["NHL"].replace({"Rangers Islanders Devils": "Rangers,Islanders,Devils",
                                           "Kings Ducks": "Kings,Ducks",
                                           "Red Wings": "Red,Wings",
                                           "Maple Leafs": "Maple,Leafs",
                                           "Blue Jackets": "Blue,Jackets",
                                           "Golden Knights": "Golden,Knights" })
    cities["NHL"] = cities["NHL"].apply(lambda x: x.split(","))
    cities = cities.explode("NHL")
    cities = cities.reset_index()
    #cities["NHL"] = cities["NHL"].str.replace("—", str(np.nan))
    nhl_df = nhl_df[nhl_df["year"] == 2018]
    nhl_df["team"] = nhl_df["team"].str.replace("*", "")
    nhl_df = nhl_df[~nhl_df["team"].str.contains("Division")]
    nhl_df["team"] = nhl_df["team"].apply(lambda x: x.split(" ")[-1])
    nhl_df = nhl_df[["team", "W", "L"]]
    nhl_df = nhl_df.astype({"W": int, "L": int, "team": str})
    nhl_df["WLR"] = nhl_df["W"] / (nhl_df["W"] + nhl_df["L"])
    cities = cities[["Metropolitan area", "Population","NHL"]]
    cities = cities.rename(columns={"NHL":"team"})
    nhl_full_df = pd.merge(cities, nhl_df, on="team", how="inner")
    
    nhl_full_df = nhl_full_df.astype({"Population": float})
    nhl_full_df = nhl_full_df.groupby("Metropolitan area").agg({"Population": np.max, "WLR": np.mean})
    print(nhl_full_df.shape)
    population_by_region = nhl_full_df["Population"] # pass in metropolitan area population from cities
    win_loss_by_region = nhl_full_df["WLR"] # pass in win/loss ratio from nhl_df in the same order as 
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    
    
nhl_correlation()

(28, 2)


PearsonRResult(statistic=0.012486162921209923, pvalue=0.9497182859911808)

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [67]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re



def nba_correlation():
    # YOUR CODE HERE
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.rename(columns = {"Population (2016 est.)[8]":"Population"},inplace=True)
    cities["NBA"] = cities["NBA"].apply(lambda x: re.sub(r"\[.+\]", "", x))
    cities["NBA"] = cities["NBA"].replace({"Knicks Nets": "Knicks,Nets",
                                           "Lakers Clippers": "Lakers,Clippers",
                                           "Trail Blazers": "Trail,Blazers"})
    cities["NBA"] = cities["NBA"].apply(lambda x: x.split(","))
    cities = cities.explode("NBA")
    cities = cities.reset_index()
    
    # cleaning nba df
    nba_df = nba_df[nba_df["year"] == 2018]
    nba_df["team"] = nba_df["team"].apply(lambda x: re.sub(r"(\*)*\s\(\d+\)", "", x))
    nba_df["team"] = nba_df["team"].apply(lambda x: x.split(" ")[-1])
    nba_df = nba_df.astype({"team": str, "W": int, "L": int})
    nba_df = nba_df[["team", "W", "L"]]
    nba_df["WLR"] = nba_df["W"] / (nba_df["W"] + nba_df["L"])
    # merging cities with nba df
    cities = cities[["Metropolitan area", "Population","NBA"]]
    cities = cities.rename(columns={"NBA":"team"})
    nba_full_df = pd.merge(cities, nba_df, on="team", how="inner")

    nba_full_df = nba_full_df.astype({"Population": float})
    nba_full_df = nba_full_df.groupby("Metropolitan area").agg({"Population": np.max, "WLR": np.mean})



    population_by_region = nba_full_df["Population"] # pass in metropolitan area population from citi
    win_loss_by_region = nba_full_df["WLR"] # pass in win/loss ratio from nhl_df in the same order as 
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nba_correlation()

-0.17657160252844614

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re



def mlb_correlation(): 
    # YOUR CODE HERE
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.rename(columns = {"Population (2016 est.)[8]":"Population"},inplace=True)
    cities["MLB"] = cities["MLB"].apply(lambda x: re.sub(r"\[.+\]", "", x))
    cities["MLB"] = cities["MLB"].replace({"Blue Jays": "Blue,Jays",
                                           "Cubs White Sox": "Cubs,White Sox",
                                           "Dodgers Angels": "Dodgers,Angels",
                                           "Giants Athletics": "Giants,Athletics",
                                           "Yankees Mets": "Yankees,Mets",
                                            "Red Sox":"Red,Sox"
                                            })
    cities["MLB"] = cities["MLB"].apply(lambda x: x.split(","))
    cities = cities.explode("MLB")
    cities = cities.reset_index()

    # cleaning the mlb_df dataframe
    mlb_df = mlb_df[mlb_df["year"] == 2018]
    mlb_df["team"] = mlb_df["team"].apply(lambda x: x.split(" ")[-1])

    mlb_df = mlb_df.astype({"team": str, "W": int, "L": int})
    mlb_df = mlb_df[["team", "W", "L"]]
    mlb_df["WLR"] = mlb_df["W"] / (mlb_df["W"] + mlb_df["L"])
    # merging cities with nba df
    cities = cities[["Metropolitan area", "Population","MLB"]]
    cities = cities.rename(columns={"MLB":"team"})
    mlb_full_df = pd.merge(cities, mlb_df, on="team", how="inner")

    # drop duplicated columns
    mlb_full_df.loc[mlb_full_df["Metropolitan area"] == "New York City", "WLR"] = 0.546 # mean of NY W-L%
    mlb_full_df.loc[mlb_full_df["Metropolitan area"] == "Los Angeles", "WLR"] = 0.5289999999999999 # mean of LA W-L%
    mlb_full_df.loc[mlb_full_df["Metropolitan area"] == "San Francisco Bay Area", "WLR"] = 0.525 # mean of SF W-L%
    mlb_full_df.loc[mlb_full_df["Metropolitan area"] == "Chicago", "WLR"] = 0.482769 # mean of CH W-L%
    mlb_full_df.loc[mlb_full_df["Metropolitan area"] == "Boston", "WLR"] = 0.666667 # mean of BO W-L%
    mlb_full_df = mlb_full_df.drop_duplicates(subset="Metropolitan area").reset_index()

    mlb_full_df = mlb_full_df.astype({"Population": float})
    mlb_full_df = mlb_full_df.groupby("Metropolitan area").agg({"Population": np.max, "WLR": np.mean})
    
    
    population_by_region = mlb_full_df["Population"] # pass in metropolitan area population from cities
    win_loss_by_region = mlb_full_df["WLR"] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]
    print(len(population_by_region))
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
mlb_correlation()

26


0.14971538811797036

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [6]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def nfl_correlation(): 
    # YOUR CODE HERE
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.rename(columns = {"Population (2016 est.)[8]":"Population"},inplace=True)
    # cleaning the cities dataframe
    cities["NFL"] = cities["NFL"].apply(lambda x: re.sub(r"\[.+\]", "", x))
    cities["NFL"] = cities["NFL"].replace({"Giants Jets": "Giants,Jets",
                                           "Rams Chargers": "Rams,Chargers",
                                           "49ers Raiders": "49ers,Raiders"
                                           })
    cities["NFL"] = cities["NFL"].apply(lambda x: x.split(","))
    cities = cities.explode("NFL")
    cities = cities.reset_index()

    # cleaning the nhl_df dataframe
    nfl_df = nfl_df[nfl_df["year"] == 2018]
    nfl_df["team"] = nfl_df["team"].apply(lambda x: re.sub(r"(\*|\+)", "", x))
    nfl_df = nfl_df[~nfl_df["team"].str.contains(r"AFC [A-Z][a-z].+", regex=True)]
    nfl_df = nfl_df[~nfl_df["team"].str.contains(r"NFC [A-Z][a-z].+", regex=True)]
    nfl_df["team"] = nfl_df["team"].apply(lambda x: x.split(" ")[-1])
    nfl_df = nfl_df.astype({"team": str, "W": int, "L": int})
    nfl_df = nfl_df[["team", "W", "L"]]
    nfl_df["WLR"] = nfl_df["W"] / (nfl_df["W"] + nfl_df["L"])
    # merging cities with nba df
    cities = cities[["Metropolitan area", "Population","NFL"]]
    #cities = cities.rename(columns={"NFL":"team"})
    nfl_full_df = pd.merge(cities, nfl_df, left_on="NFL", right_on="team")
    nfl_full_df = nfl_full_df.astype({"Population": float})

    # Population and win/loss correlation
    nfl_full_df = nfl_full_df.groupby("Metropolitan area").agg({"Population": np.sum, "WLR": np.mean})
    
    population_by_region = nfl_full_df["Population"] # pass in metropolitan area population from cities
    win_loss_by_region = nfl_full_df["WLR"] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"
   

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nfl_correlation()

-0.06283512354755244

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [29]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

#nhl
def nhl_cleaned_df() -> pd.DataFrame:
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.rename(columns = {"Population (2016 est.)[8]":"Population"},inplace=True)
    #cities["NHL"] = cities["NHL"].str.replace("—", str(np.nan))
    cities["NHL"] = cities["NHL"].apply(lambda x: re.sub(r"\[.+\]", "", x))
    cities["NHL"] = cities["NHL"].replace({"Rangers Islanders Devils": "Rangers,Islanders,Devils",
                                           "Kings Ducks": "Kings,Ducks",
                                           "Red Wings": "Red,Wings",
                                           "Maple Leafs": "Maple,Leafs",
                                           "Blue Jackets": "Blue,Jackets",
                                           "Golden Knights": "Golden,Knights" })
    cities["NHL"] = cities["NHL"].apply(lambda x: x.split(","))
    cities = cities.explode("NHL")
    cities = cities.reset_index()
    #cities["NHL"] = cities["NHL"].str.replace("—", str(np.nan))
    nhl_df = nhl_df[nhl_df["year"] == 2018]
    nhl_df["team"] = nhl_df["team"].str.replace("*", "")
    nhl_df = nhl_df[~nhl_df["team"].str.contains("Division")]
    nhl_df["team"] = nhl_df["team"].apply(lambda x: x.split(" ")[-1])
    nhl_df = nhl_df[["team", "W", "L"]]
    nhl_df = nhl_df.astype({"W": int, "L": int, "team": str})
    nhl_df["WLR"] = nhl_df["W"] / (nhl_df["W"] + nhl_df["L"])
    cities = cities[["Metropolitan area", "Population","NHL"]]
    cities = cities.rename(columns={"NHL":"team"})
    nhl_full_df = pd.merge(cities, nhl_df, on="team", how="inner")

    nhl_full_df = nhl_full_df.astype({"Population": float})
    return nhl_full_df
# nba
def nba_cleaned_df() -> pd.DataFrame:
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.rename(columns = {"Population (2016 est.)[8]":"Population"},inplace=True)
    cities["NBA"] = cities["NBA"].apply(lambda x: re.sub(r"\[.+\]", "", x))
    cities["NBA"] = cities["NBA"].replace({"Knicks Nets": "Knicks,Nets",
                                           "Lakers Clippers": "Lakers,Clippers",
                                           "Trail Blazers": "Trail,Blazers"})
    cities["NBA"] = cities["NBA"].apply(lambda x: x.split(","))
    cities = cities.explode("NBA")
    cities = cities.reset_index()

    # cleaning nba df
    nba_df = nba_df[nba_df["year"] == 2018]
    nba_df["team"] = nba_df["team"].apply(lambda x: re.sub(r"(\*)*\s\(\d+\)", "", x))
    nba_df["team"] = nba_df["team"].apply(lambda x: x.split(" ")[-1])
    nba_df = nba_df.astype({"team": str, "W": int, "L": int})
    nba_df = nba_df[["team", "W", "L"]]
    nba_df["WLR"] = nba_df["W"] / (nba_df["W"] + nba_df["L"])
    # merging cities with nba df
    cities = cities[["Metropolitan area", "Population","NBA"]]
    cities = cities.rename(columns={"NBA":"team"})
    nba_full_df = pd.merge(cities, nba_df, on="team", how="inner")

    nba_full_df = nba_full_df.astype({"Population": float})
    return nba_full_df
#mlb
def mlb_cleaned_df() -> pd.DataFrame:
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.rename(columns = {"Population (2016 est.)[8]":"Population"},inplace=True)
    cities["MLB"] = cities["MLB"].apply(lambda x: re.sub(r"\[.+\]", "", x))
    cities["MLB"] = cities["MLB"].replace({"Blue Jays": "Blue,Jays",
                                           "Cubs White Sox": "Cubs,White Sox",
                                           "Dodgers Angels": "Dodgers,Angels",
                                           "Giants Athletics": "Giants,Athletics",
                                           "Yankees Mets": "Yankees,Mets",
                                           "Red Sox":"Red,Sox"
                                           })
    cities["MLB"] = cities["MLB"].apply(lambda x: x.split(","))
    cities = cities.explode("MLB")
    cities = cities.reset_index()

    # cleaning the mlb_df dataframe
    mlb_df = mlb_df[mlb_df["year"] == 2018]
    mlb_df["team"] = mlb_df["team"].apply(lambda x: x.split(" ")[-1])

    mlb_df = mlb_df.astype({"team": str, "W": int, "L": int})
    mlb_df = mlb_df[["team", "W", "L"]]
    mlb_df["WLR"] = mlb_df["W"] / (mlb_df["W"] + mlb_df["L"])
    # merging cities with nba df
    cities = cities[["Metropolitan area", "Population","MLB"]]
    cities = cities.rename(columns={"MLB":"team"})
    mlb_full_df = pd.merge(cities, mlb_df, on="team", how="inner")

    # drop duplicated columns
    mlb_full_df.loc[mlb_full_df["Metropolitan area"] == "New York City", "WLR"] = 0.546 # mean of NY W-L%
    mlb_full_df.loc[mlb_full_df["Metropolitan area"] == "Los Angeles", "WLR"] = 0.5289999999999999 # mean of LA W-L%
    mlb_full_df.loc[mlb_full_df["Metropolitan area"] == "San Francisco Bay Area", "WLR"] = 0.525 # mean of SF W-L%
    mlb_full_df.loc[mlb_full_df["Metropolitan area"] == "Chicago", "WLR"] = 0.482769 # mean of CH W-L%
    mlb_full_df.loc[mlb_full_df["Metropolitan area"] == "Boston", "WLR"] = 0.666667 # mean of BO W-L%
    mlb_full_df = mlb_full_df.drop_duplicates(subset="Metropolitan area").reset_index()

    mlb_full_df = mlb_full_df.astype({"Population": float})
    return mlb_full_df
# nfl
def nfl_cleaned_df() -> pd.DataFrame:
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.rename(columns = {"Population (2016 est.)[8]":"Population"},inplace=True)
    # cleaning the cities dataframe
    cities["NFL"] = cities["NFL"].apply(lambda x: re.sub(r"\[.+\]", "", x))
    cities["NFL"] = cities["NFL"].replace({"Giants Jets": "Giants,Jets",
                                           "Rams Chargers": "Rams,Chargers",
                                           "49ers Raiders": "49ers,Raiders"
                                           })
    cities["NFL"] = cities["NFL"].apply(lambda x: x.split(","))
    cities = cities.explode("NFL")
    cities = cities.reset_index()

    # cleaning the nhl_df dataframe
    nfl_df = nfl_df[nfl_df["year"] == 2018]
    nfl_df["team"] = nfl_df["team"].apply(lambda x: re.sub(r"(\*|\+)", "", x))
    nfl_df = nfl_df[~nfl_df["team"].str.contains(r"AFC [A-Z][a-z].+", regex=True)]
    nfl_df = nfl_df[~nfl_df["team"].str.contains(r"NFC [A-Z][a-z].+", regex=True)]
    nfl_df["team"] = nfl_df["team"].apply(lambda x: x.split(" ")[-1])
    nfl_df = nfl_df.astype({"team": str, "W": int, "L": int})
    nfl_df = nfl_df[["team", "W", "L"]]
    nfl_df["WLR"] = nfl_df["W"] / (nfl_df["W"] + nfl_df["L"])
    # merging cities with nba df
    cities = cities[["Metropolitan area", "Population","NFL"]]
    #cities = cities.rename(columns={"NFL":"team"})
    nfl_full_df = pd.merge(cities, nfl_df, left_on="NFL", right_on="team")
    nfl_full_df = nfl_full_df.astype({"Population": float})
    
    return  nfl_full_df

# match which df
def league_df(sport):
    nfl_full_df = nfl_cleaned_df()[["Metropolitan area", "WLR"]]
    nhl_full_df = nhl_cleaned_df()[["Metropolitan area", "WLR"]]
    mlb_full_df = mlb_cleaned_df()[["Metropolitan area", "WLR"]]
    nba_full_df = nba_cleaned_df()[["Metropolitan area", "WLR"]]
    if sport =='NFL':
        return nfl_full_df
    elif sport =='NBA':
        return nba_full_df
    elif sport =='NHL':
        return nhl_full_df
    elif sport =='MLB':
        return mlb_full_df
    else:
        print("We're only interested in Big4!.")
# p-value
def evaluate_p_values(Big4: list, func):
    p_values = pd.DataFrame({k:np.nan for k in Big4}, index=Big4)
    for i in Big4:
        for j in Big4:
            if i != j:
                big4_df = pd.merge(func(i), func(j), on="Metropolitan area", how="inner")
                p_values.loc[i, j] = stats.ttest_rel(big4_df["WLR_x"], big4_df["WLR_y"])[1]
    return p_values


    
def sports_team_performance():
    # YOUR CODE HERE    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan

    
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values_dict = evaluate_p_values(sports, league_df)
    p_values = pd.DataFrame(p_values_dict).astype("float")
    #p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values
sports_team_performance()

AssertionError: The NBA-NHL p-value should be around 0.02

In [30]:
sports = ['NFL', 'NBA', 'NHL', 'MLB']
p_values_dict = evaluate_p_values(sports, league_df)
p_values = pd.DataFrame(p_values_dict)
p_values

,NFL,NBA,NHL,MLB
NFL,NaN,0.768652,0.007585,0.962810
NBA,0.768652,NaN,0.000076,0.755916
NHL,0.007585,0.000076,NaN,0.001293
MLB,0.962810,0.755916,0.001293,NaN


In [31]:
print(p_values.loc["NHL", "NBA"])
print(p_values.loc["NFL", "MLB"])

7.574348978012254e-05
0.9628096337170121


In [22]:
p_values_dict

,NFL,NBA,NHL,MLB
NFL,1.0,0.768652,0.007585,0.96281
NBA,0.768652,1.0,0.000076,0.755916
NHL,0.007585,0.000076,1.0,0.001293
MLB,0.96281,0.755916,0.001293,NaN


In [17]:
nfl_full_df = nfl_cleaned_df()[["Metropolitan area", "WLR"]]
nhl_full_df = nhl_cleaned_df()[["Metropolitan area", "WLR"]]
df = pd.merge(nfl_full_df, nhl_full_df, on="Metropolitan area")
df

,Metropolitan area,WLR_x,WLR_y
0,New York City,0.312500,0.465753
1,New York City,0.312500,0.486111
2,New York City,0.312500,0.602740
3,New York City,0.250000,0.465753
4,New York City,0.250000,0.486111
5,New York City,0.250000,0.602740
6,Los Angeles,0.812500,0.608108
7,Los Angeles,0.812500,0.637681
8,Los Angeles,0.750000,0.608108
9,Los Angeles,0.750000,0.637681


In [20]:
p = stats.ttest_rel(df["WLR_x"], df["WLR_y"])
p

TtestResult(statistic=-2.904580663852721, pvalue=0.0075847697991172245, df=25)